In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, Imputer
from sklearn.model_selection import train_test_split

from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import optimizers
from keras import callbacks

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [8]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#y_airfoil = np.reshape(y_airfoil.values, (-1,1))
#Wdbc
x_wdbc = df_wdbc[['radius_mean','perimeter_mean','area_mean','concavity_mean','concave points_mean','radius_worst','perimeter_worst','area_worst','concave points_worst']]
y_wdbc = df_wdbc['diagnosis']
labelencoder_Y = LabelEncoder()
y_wdbc = labelencoder_Y.fit_transform(y_wdbc)
#Automobile
x_automobile = df_automobile.drop(['price','make','body-style'], axis = 1)
y_automobile = df_automobile['price']
y_automobile = np.reshape(y_automobile.values, (-1,1))
#Autompg
x_autompg = df_autompg.drop(['Mpg','Car Name'], axis = 1)
y_autompg = df_autompg['Mpg']
#Challenger
x_challenger = df_challenger.drop(['Number of O-rings','Temporal Order of Flight'], axis = 1)
y_challenger = df_challenger['Number of O-rings']

In [9]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_automobile, x_test_automobile, y_train_automobile, y_test_automobile = train_test_split1(x_automobile, y_automobile)
x_train_autompg, x_test_autompg, y_train_autompg, y_test_autompg = train_test_split1(x_autompg, y_autompg)
x_train_challenger, x_test_challenger, y_train_challenger, y_test_challenger = train_test_split1(x_challenger, y_challenger)
x_train_wdbc, x_test_wdbc, y_train_wdbc, y_test_wdbc = train_test_split1(x_wdbc, y_wdbc)

In [10]:
def train_val_split(x_train, y_train):
    X_train, X_val, Y_train, Y_val= train_test_split(x_train, y_train, test_size=0.176, random_state=42)
    return X_train, X_val, Y_train, Y_val

In [11]:
def standard_scaler(X_train, X_val, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    X_scale_val=sc.fit(X_val)
    x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    return X_train, X_val, x_test, sc

In [12]:
def min_max_scaler(X_train, X_val, x_test, Y_train, Y_val, y_test):
    min_max=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=min_max.fit(X_train)
    X_scale_val=min_max.fit(X_val)
    x_scale_test=min_max.fit(x_test)
    Y_scale_train=min_max.fit(Y_train)
    Y_scale_val=min_max.fit(Y_val)
    y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=y_scale_test.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, min_max

In [13]:
#Optimizer list
adam0 = 'adam'
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.005, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.001,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil = train_val_split(x_train_airfoil, y_train_airfoil)
X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile = train_val_split(x_train_automobile, y_train_automobile)
X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg = train_val_split(x_train_autompg, y_train_autompg)
X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger = train_val_split(x_train_challenger, y_train_challenger)
X_train_wdbc, X_val_wdbc, Y_train_wdbc, Y_val_wdbc = train_val_split(x_train_wdbc, y_train_wdbc)

In [15]:
#X_train_airfoil, X_val_airfoil, x_test_airfoil, Y_train_airfoil, Y_val_airfoil, y_test_airfoil, min_max_airfoil=min_max_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil, Y_train_airfoil, Y_val_airfoil, y_test_airfoil)
X_train_airfoil, X_val_airfoil, x_test_airfoil, sc_airfoil=standard_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil)
X_train_wdbc, X_val_wdbc, x_test_wdbc, sc_wdbc=standard_scaler(X_train_wdbc, X_val_wdbc, x_test_wdbc)
#X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile = normalization(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile, min_max_automobile=min_max_scaler(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_autompg, X_val_autompg, x_test_autompg, sc_autompg=standard_scaler(X_train_autompg, X_val_autompg, x_test_autompg)
X_train_challenger, X_val_challenger, x_test_challenger, sc_challenger=standard_scaler(X_train_challenger, X_val_challenger, x_test_challenger)

In [16]:
X_train_airfoil

array([[ 0.40429466, -0.43865897, -0.90708716,  1.49006077, -0.74870942],
       [-0.66808819, -0.58768726, -0.36689823,  1.49006077, -0.70484024],
       [-0.95405694,  0.07466068,  0.98357409, -1.09643364,  0.11643575],
       ...,
       [ 0.10045285, -0.23995459,  0.1732907 ,  0.45807563, -0.53642977],
       [-0.45361162,  1.41591525, -0.90708716, -0.58044107,  1.21196682],
       [ 0.10045285,  0.33959985, -0.36689823,  1.49006077, -0.09974605]])

In [16]:
df_airfoil=preprocessing_data1(df_airfoil,5)
df_automobile=preprocessing_data1(df_automobile,22)
df_autompg=preprocessing_data1(df_autompg,7)
df_challenger=preprocessing_data1(df_challenger,3)
df_combine=preprocessing_data1(df_combine,4)
df_computer=preprocessing_data1(df_computer,9)
df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
df_concrete1=preprocessing_data1(df_concrete1,7)
df_concrete2=preprocessing_data1(df_concrete2,7)
df_concrete3=preprocessing_data1(df_concrete3,7)
df_energy1=preprocessing_data1(df_energy1,8)
df_energy2=preprocessing_data1(df_energy2,8)

df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
df_housing=preprocessing_data1(df_housing,13)

df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

NameError: name 'df_combine' is not defined

In [17]:
def auto_encoder(X_train, X_val, x_test, encoded_dim, size, epochs):

    #Input dimension

    ncols=X_train.shape[1]
    input_dim=Input(shape=(ncols, ))

    #Encoder

    encoded1=Dense(2000, activation='relu')(input_dim)
    encoded2=Dense(1500, activation='relu')(encoded1)
    encoded3=Dense(1000, activation='relu')(encoded2)
    encoded4=Dense(500, activation='relu')(encoded3)
    encoded5=Dense(100, activation='relu')(encoded4)
    encoded6=Dense(50, activation='relu')(encoded5)
    encoded7=Dense(encoded_dim, activation='relu')(encoded6)
    
    #Decoder
    
    decoded1=Dense(50, activation='relu')(encoded7)
    decoded2=Dense(100, activation='relu')(decoded1)
    decoded3=Dense(500, activation='relu')(decoded2)
    decoded4=Dense(1000, activation='relu')(decoded3)
    decoded5=Dense(1500, activation='relu')(decoded4)
    decoded6=Dense(2000, activation='relu')(decoded5)
    decoded7=Dense(ncols, activation='relu')(decoded6)
    
    #Auto encoder
    
    auto_encoder=Model(inputs = input_dim, outputs = decoded7)
    auto_encoder.summary()
    auto_encoder.compile(optimizer=adam0, loss='mse')
    auto_encoder.fit(X_train,X_train, batch_size=size,epochs=epochs, validation_data=(X_val, X_val), shuffle=True)
    evaluate = auto_encoder.evaluate(x_test, x_test)
    print(evaluate)
    
    #Encoder network
    encoder=Model(inputs=input_dim, outputs=encoded7)
    
    return auto_encoder, evaluate, encoder

In [18]:
auto_encoder_airfoil, test_error, encoder_airfoil=auto_encoder(X_train_airfoil, X_val_airfoil, x_test_airfoil, 3, 32, 20)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 2000)              12000     
_________________________________________________________________
dense_2 (Dense)              (None, 1500)              3001500   
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              1501000   
_________________________________________________________________
dense_4 (Dense)              (None, 500)               500500    
_________________________________________________________________
dense_5 (Dense)              (None, 100)               50100     
_________________________________________________________________
dense_6 (Dense)              (None, 50)                5050      
__________

In [19]:
pred=auto_encoder_airfoil.predict(x_test_airfoil)
pred=sc_airfoil.inverse_transform(pred)
actual=sc_airfoil.inverse_transform(x_test_airfoil)
actual_y=list(actual)
for i in range(len(actual_y)):
    print('actual', actual_y[i], 'prediction', pred[i])

actual [4.0000000e+02 8.8817842e-16 3.0480000e-01 3.1700000e+01 3.3126600e-03] prediction [2.86898242e+03 6.84911489e+00 3.07068944e-01 4.84867249e+01
 1.16738025e-02]
actual [1.25000e+03 1.23000e+01 1.01600e-01 3.17000e+01 4.18756e-02] prediction [2.86898242e+03 6.84911489e+00 1.36103541e-01 4.84867249e+01
 1.16738025e-02]
actual [2.50000e+03 4.00000e+00 3.04800e-01 3.96000e+01 5.79636e-03] prediction [2.86898242e+03 6.84911489e+00 3.08325559e-01 4.84867249e+01
 1.16738025e-02]
actual [4.0000000e+03 8.8817842e-16 2.5400000e-02 3.1700000e+01 4.3947200e-04] prediction [2.86898242e+03 6.84911489e+00 1.36103541e-01 4.84867249e+01
 1.16738025e-02]
actual [5.0000000e+03 8.8817842e-16 5.0800000e-02 5.5500000e+01 7.6193000e-04] prediction [2.86898242e+03 6.84911489e+00 1.36103541e-01 5.56287689e+01
 1.16738025e-02]
actual [8.00000e+02 4.80000e+00 2.54000e-02 3.96000e+01 9.07475e-04] prediction [2.86898242e+03 6.84911489e+00 1.36103541e-01 4.84867249e+01
 1.16738025e-02]
actual [1.00000e+03 1.

In [ ]:
def NN_model_structure(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate